# Uses interaction to push triples to the brain and query it

This notebook assumes you already understand the following notebooks:

* lets-chat.ipynb
* roboGrasp-api.ipynb

In this notebook, we will combine the interaction modeled through EMISSOR with the interaction throught *capsules* with the BRAIN. As auxiliary modules, we will use the *cltl-knowledgeextraction* and *cltl-language-generation*. The *cltl-knowledgeextraction* will extract triples from the utterances either for posting or for querying. In the former case, it also extracts the source perspective from the text. The triples and source perspectives are represented in enriched capsules. We will also use a replier that is include in the *cltl-languagegeneration* package. This replier transfers the response from the BRAIN into natural language expressions and possibly gestures. 

In order to connect to the EMISSOR scenario, we need to align the meta properties of the scenario with the meta data in the capsules. However,  the *cltl-knowledgeextraction* uses a similar data object to keep track of the conversation history. This is a Chat object that needs to be created. Through this Chat object, we keep track of what was said before and deal with coreference to earlier utterances.

Combining EMISSOR and the BRAIN, we can model the interaction in a infite while loop where we go through the following steps:

* We create an EMISSOR scenario at the start of the interaction and a corresponding Chat object to keep track of the dialogue history
* while the user does not stop:
    * create TextSignals for each utterance from the user and store these in an EMISSOR scenario
    * add the utterance also to the Chat instance
    * process the latest utterance in the Chat object using the *cltl-knowledgeextraction* to get triples and perspectives
    * create a capsule from the triples, perspectives and the text signal meta data
    * post the triple to the BRAIN
    * get the answer or throughts as a response from the BRAIN
    * verbalise the answer or thoughts
    * create a new TextSignal for the system response and add it to the EMISSOR scenario
* When the user stops, we save the scenario in the EMISSOR format.



Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com


## Import the necessary modules

In [2]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

### Import the chatbot utility functions

In [3]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Specifying the BRAIN

We imported the brain from CLTL. We create an instance using the class *LongTermMemory*, which takes three parameters: 1) the address of the triple store, 2) a path to a folder for logging the triples and a boolean flag for clearing the repository and reload the initial ontologies.

As the address of the triple store, we give the GraphDB localhost port (7200) and specify the name of the repository that we created beforehand. Any repository will do. We use here the *sandbox* repository. Note that you can also specify a remote SPARQL endpoint of another triple store or share a triple store among systems.

For storing the triples generate, we define a scenario folder in a data folder relative to where the notebooks are stored. It is based on the timestamp when we start. We will use this scenario structure later for the interaction as well as we did before.

Finall, if you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [4]:
### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)
    
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

replier = LenkaReplier()
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-11-11 10:46:47,124 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-11 10:46:50,088 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Standard initialisation of a scenario

In the next part, we will integrate calling the brain with the EMISSOR scenarios. This means that we can use the meta data from the EMSSIOR scenario when we create a capsule. We therefore first do the standard initialisation for EMISSOR to start a scenario.

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-04-17:39:09  Created 
Directory  ../../data/2021-11-04-17:39:09/image  Created 


In [7]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario, AGENT)

chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply_(test_triple, UtteranceType.QUESTION, scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: How's it going? What is your name? Piek?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


2021-11-05 12:00:46,926 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     000 - << Start of Chat with Piek_t_926000 >>
Leolani2: Would you like to chat? I'll do my best to keep up



 What is your name


Piek: What is your name
2021-11-05 12:01:34,222 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     001 - Piek_t_926000: "What is your name"
UtteranceType.QUESTION
{'predicate': {'label': 'name-is', 'type': ['noun.communication', 'verb.stative']}, 'subject': {'label': 'leolani', 'type': ['agent']}, 'object': {'label': '', 'type': []}}
2021-11-05 12:01:34,228 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: leolani_like_? [agent_->_])
Leolani2: I wouldn't know!


KeyboardInterrupt: Interrupted by user

NameError: name 'answer_a_query' is not defined

In [8]:
scenario.scenario.ruler.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook